In [1]:
import os 
import pandas as pd
import numpy as np

# import sklearn methods 
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
import sys 
from sklearn.model_selection import learning_curve
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier

In [2]:
# display test scores and return result string and indexes of false samples
def display_test_scores(test, pred):
    str_out = ""
    str_out += ("TEST SCORES\n")
    str_out += ("\n")

    #print accuracy
    accuracy = accuracy_score(test, pred)
    str_out += ("ACCURACY: {:.4f}\n".format(accuracy))
    str_out += ("\n")


    #print confusion matrix
    str_out += ("CONFUSION MATRIX:\n")
    conf_mat = confusion_matrix(test, pred)
    str_out += ("{}".format(conf_mat))
    str_out += ("\n")
    str_out += ("\n")

    #print FP, FN
    str_out += ("FALSE POSITIVES:\n")
    fp = conf_mat[1][0]
    pos_labels = conf_mat[1][0]+conf_mat[1][1]
    str_out += ("{} out of {} positive labels ({:.4f}%)\n".format(fp, pos_labels,fp/pos_labels))
    str_out += ("\n")

    str_out += ("FALSE NEGATIVES:\n")
    fn = conf_mat[0][1]
    neg_labels = conf_mat[0][1]+conf_mat[0][0]
    str_out += ("{} out of {} negative labels ({:.4f}%)\n".format(fn, neg_labels, fn/neg_labels))
    str_out += ("\n")

    #print classification report
    str_out += ("PRECISION, RECALL, F1 scores:\n")
    str_out += ("{}".format(classification_report(test, pred)))
    
    false_indexes = np.where(test != pred)
    return str_out, false_indexes

In [3]:
################ not scaled dataset ###################
df_original = pd.read_pickle('../input/rlddandmerged/rldd_normalized_scaled.pkl')

X = df_original.loc[:, ["n_EAR", 
                    "n_MAR", "n_MOE", "n_EC",
                    "n_LEB", "n_SOP", "PERCLOS", "CLOSENESS"]]


y = df_original.loc[:, "DROWSINESS"]
y = y.map({0: 0, 0.5: 1, 1:2})
df_original.loc[:, "DROWSINESS"] = y

y = df_original.loc[:, "DROWSINESS"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [4]:
# cross-validation with 5 splits
cv = StratifiedShuffleSplit(n_splits=10, random_state = 42)

# CART decision tree
bagging = BaggingClassifier(random_state=0)

# parameters 
parameters = {
                #"max_features": [0.3, 1],
                #"max_samples": [0.3, 1],
                #"n_estimators": [10, 100]
                }

In [5]:
# grid search for parameters
grid = GridSearchCV(estimator=bagging, param_grid=parameters, cv=cv, n_jobs=-1)
grid.fit(X_train, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid.best_params_, grid.best_score_))

# prediction results
y_pred = grid.predict(X_test)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)

The best parameters are {} with a score of 0.8646
TEST SCORES

ACCURACY: 0.8672

CONFUSION MATRIX:
[[178338  11866   8680]
 [ 19557 166892  11871]
 [ 14612  13104 175177]]

FALSE POSITIVES:
19557 out of 186449 positive labels (0.1049%)

FALSE NEGATIVES:
11866 out of 190204 negative labels (0.0624%)

PRECISION, RECALL, F1 scores:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87    198884
           1       0.87      0.84      0.86    198320
           2       0.90      0.86      0.88    202893

    accuracy                           0.87    600097
   macro avg       0.87      0.87      0.87    600097
weighted avg       0.87      0.87      0.87    600097

